In [ ]:
#VGG16 Base Model

In [59]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [60]:
train_dir = 'C:/Users/Kranti/Desktop/EE541_project/archive/Training'
val_dir = 'C:/Users/Kranti/Desktop/EE541_project/archive/Validation'

In [61]:
# Print the number of images in each category for the training set
print('Training set:')
for class_name in os.listdir(train_dir):
    class_dir = os.path.join(train_dir, class_name)
    print(class_name + ':', len(os.listdir(class_dir)))

# Print the number of images in each category for the validation set
print('Validation set:')
for class_name in os.listdir(val_dir):
    class_dir = os.path.join(val_dir, class_name)
    print(class_name + ':', len(os.listdir(class_dir)))

Training set:
notumor: 1600
tumor: 4018
Validation set:
notumor: 400
tumor: 1005


In [53]:
batch_size = 32
epochs = 10

# Create an instance of the VGG16 model
vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers in the VGG16 model
for layer in vgg.layers:
    layer.trainable = False

# Create a new model that includes the VGG16 model and additional layers
model = Sequential()
model.add(vgg)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Create generators for the training and validation data
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=batch_size, class_mode='binary')
val_generator = val_datagen.flow_from_directory(val_dir, target_size=(224, 224), batch_size=batch_size, class_mode='binary')

# Train the model
model.fit(train_generator, epochs=epochs, validation_data=val_generator)

# Calculate the validation accuracy
val_loss, val_acc = model.evaluate(val_generator)
print('Validation accuracy:', val_acc)

Found 5618 images belonging to 2 classes.
Found 1405 images belonging to 2 classes.
Epoch 1/10
176/176 [==============================] - 630s 4s/step - loss: 0.2053 - accuracy: 0.9514 - val_loss: 0.0453 - val_accuracy: 0.9879
Epoch 2/10
176/176 [==============================] - 597s 3s/step - loss: 0.0333 - accuracy: 0.9902 - val_loss: 0.0392 - val_accuracy: 0.9872
Epoch 3/10
176/176 [==============================] - 2157s 12s/step - loss: 0.0147 - accuracy: 0.9954 - val_loss: 0.0152 - val_accuracy: 0.9929
Epoch 4/10
176/176 [==============================] - 662s 4s/step - loss: 0.0042 - accuracy: 0.9995 - val_loss: 0.0136 - val_accuracy: 0.9957
Epoch 5/10
176/176 [==============================] - 625s 4s/step - loss: 0.0046 - accuracy: 0.9993 - val_loss: 0.0094 - val_accuracy: 0.9964
Epoch 6/10
176/176 [==============================] - 620s 4s/step - loss: 0.0018 - accuracy: 0.9996 - val_loss: 0.0102 - val_accuracy: 0.9950
Epoch 7/10
176/176 [==============================] - 63

In [55]:
test_dir = 'C:/Users/Kranti/Desktop/EE541_project/archive/Testing'
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary')

# Evaluate the model on the test set
scores = model.evaluate(test_generator)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Found 253 images belonging to 2 classes.
8/8 [==============================] - 16s 2s/step - loss: 1.6747 - accuracy: 0.6759
Test loss: 1.6746888160705566
Test accuracy: 0.6758893132209778


In [ ]:
#PyTorch Version

In [56]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

# Set the directories for the training, validation, and test sets
train_dir = 'C:/Users/Kranti/Desktop/EE541_project/archive/Training'
val_dir = 'C:/Users/Kranti/Desktop/EE541_project/archive/Validation'
test_dir = 'C:/Users/Kranti/Desktop/EE541_project/archive/Testing'

# Print the number of images in each category for the training set
print('Training set:')
for class_name in os.listdir(train_dir):
    class_dir = os.path.join(train_dir, class_name)
    print(class_name + ':', len(os.listdir(class_dir)))

# Print the number of images in each category for the validation set
print('Validation set:')
for class_name in os.listdir(val_dir):
    class_dir = os.path.join(val_dir, class_name)
    print(class_name + ':', len(os.listdir(class_dir)))

# Define the batch size, number of epochs, and learning rate
batch_size = 32
epochs = 10
learning_rate = 0.001

# Define the transformations for the training, validation, and test sets
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create data loaders for the training, validation, and test sets
train_dataset = ImageFolder(train_dir, transform=train_transforms)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = ImageFolder(val_dir, transform=val_transforms)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataset = ImageFolder(test_dir, transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

# Create an instance of the VGG16 model and replace the last layer with a new one
vgg = models.vgg16(pretrained=True)
for param in vgg.parameters():
    param.requires_grad = False
vgg.classifier[-1] = nn.Linear(4096, 1)

# Define the loss function, optimizer, and scheduler
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(vgg.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Train the model
for epoch in range(epochs):
    running_loss = 0.0
    running_corrects = 0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = vgg(inputs)
        loss = criterion(outputs.view(-1), targets.float())
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        preds = torch.round(outputs)
        running_corrects += torch.sum(preds == targets)

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects / len(train_dataset)

    # Evaluate the model on the validation set
    val_loss = 0.0
    val_acc = 0.0
    num_batches = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = vgg(inputs)
            loss = criterion(outputs.view(-1), targets.float())

            val_loss += loss.item() * inputs.size(0)
            preds = torch.round(outputs)
            val_acc += torch.sum(preds == targets)
            num_batches += 1

    val_loss /= len(val_dataset)
    val_acc /= len(val_dataset)

    print('Epoch [{}/{}], train_loss: {:.4f}, train_acc: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}'.format(
        epoch+1, epochs, epoch_loss, epoch_acc, val_loss, val_acc))

Training set:
notumor: 1600
tumor: 4018
Validation set:
notumor: 400
tumor: 1005


C:\Users\Kranti\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Kranti\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\Kranti/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100%|███████████████████████████████████████████████████████████████████████████████| 528M/528M [00:22<00:00, 24.3MB/s]


Epoch [1/10], train_loss: 0.1713, train_acc: 2.1011, val_loss: 0.1092, val_acc: 1.0833
Epoch [2/10], train_loss: 0.1108, train_acc: 0.9151, val_loss: 0.0915, val_acc: 1.0149
Epoch [3/10], train_loss: 0.1003, train_acc: 0.8099, val_loss: 0.0802, val_acc: 0.7829
Epoch [4/10], train_loss: 0.0975, train_acc: 0.7763, val_loss: 0.0813, val_acc: 0.7537
Epoch [5/10], train_loss: 0.0978, train_acc: 0.7786, val_loss: 0.0833, val_acc: 0.5374
Epoch [6/10], train_loss: 0.0831, train_acc: 0.6433, val_loss: 0.0871, val_acc: 0.7886
Epoch [7/10], train_loss: 0.0783, train_acc: 0.7147, val_loss: 0.0715, val_acc: 0.5701
Epoch [8/10], train_loss: 0.0802, train_acc: 0.6451, val_loss: 0.0664, val_acc: 0.6320
Epoch [9/10], train_loss: 0.0785, train_acc: 0.6670, val_loss: 0.0743, val_acc: 0.6320
Epoch [10/10], train_loss: 0.0790, train_acc: 0.5643, val_loss: 0.0817, val_acc: 0.6107


In [62]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader

train_dir = 'C:/Users/Kranti/Desktop/EE541_project/archive/Training'
val_dir = 'C:/Users/Kranti/Desktop/EE541_project/archive/Validation'

# Print the number of images in each category for the training set
print('Training set:')
for class_name in os.listdir(train_dir):
    class_dir = os.path.join(train_dir, class_name)
    print(class_name + ':', len(os.listdir(class_dir)))

# Print the number of images in each category for the validation set
print('Validation set:')
for class_name in os.listdir(val_dir):
    class_dir = os.path.join(val_dir, class_name)
    print(class_name + ':', len(os.listdir(class_dir)))

batch_size = 32
epochs = 10

# Create an instance of the VGG16 model
vgg = models.vgg16(pretrained=True)

# Freeze the layers in the VGG16 model
for param in vgg.parameters():
    param.requires_grad = False

# Create a new model that includes the VGG16 model and additional layers
model = nn.Sequential(
    vgg.features,
    nn.Flatten(),
    nn.Linear(25088, 256),
    nn.ReLU(),
    nn.Linear(256, 1),
    nn.Sigmoid()
)

# Move the model to the GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define the loss function, optimizer, and scheduler
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Create transforms for the training and validation data
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Create datasets for the training and validation data
train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
val_dataset = datasets.ImageFolder(val_dir, transform=val_transforms)

# Create data loaders for the training and validation data
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Train the model
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.float().unsqueeze(1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        predicted = outputs > 0.5
        total += labels.size(0)
        correct += (predicted == labels.float().unsqueeze(1)).sum().item()
    train_loss = running_loss / len(train_dataset)
    train_acc = correct / total

    # Evaluate the model on the validation set
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels.float().unsqueeze(1))
            val_loss += loss.item() * inputs.size(0)
            predicted = outputs > 0.5
            total += labels.size(0)
            correct += (predicted == labels.float().unsqueeze(1)).sum().item()
    val_loss = val_loss / len(val_dataset)
    val_acc = correct / total

    print(f'Epoch {epoch+1}/{epochs} -- Training Loss: {train_loss:.4f} -- Training Accuracy: {train_acc:.4f} -- Validation Loss: {val_loss:.4f} -- Validation Accuracy: {val_acc:.4f}')

# Evaluate the model on the test set
model.eval()
test_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels.float().unsqueeze(1))
        test_loss += loss.item() * inputs.size(0)
        predicted = outputs > 0.5
        total += labels.size(0)
        correct += (predicted == labels.float().unsqueeze(1)).sum().item()
test_loss = test_loss / len(test_dataset)
test_acc = correct / total
print(f'Test Loss: {test_loss:.4f} -- Test Accuracy: {test_acc:.4f}')


Training set:
notumor: 1600
tumor: 4018
Validation set:
notumor: 400
tumor: 1005
Epoch 1/10 -- Training Loss: 0.1841 -- Training Accuracy: 0.9299 -- Validation Loss: 0.0497 -- Validation Accuracy: 0.9843
Epoch 2/10 -- Training Loss: 0.0997 -- Training Accuracy: 0.9640 -- Validation Loss: 0.0383 -- Validation Accuracy: 0.9865
Epoch 3/10 -- Training Loss: 0.0807 -- Training Accuracy: 0.9717 -- Validation Loss: 0.0388 -- Validation Accuracy: 0.9843
Epoch 4/10 -- Training Loss: 0.0833 -- Training Accuracy: 0.9703 -- Validation Loss: 0.0318 -- Validation Accuracy: 0.9872
Epoch 5/10 -- Training Loss: 0.0722 -- Training Accuracy: 0.9758 -- Validation Loss: 0.0360 -- Validation Accuracy: 0.9872
Epoch 6/10 -- Training Loss: 0.0706 -- Training Accuracy: 0.9754 -- Validation Loss: 0.0313 -- Validation Accuracy: 0.9886
Epoch 7/10 -- Training Loss: 0.0643 -- Training Accuracy: 0.9781 -- Validation Loss: 0.0196 -- Validation Accuracy: 0.9922
Epoch 8/10 -- Training Loss: 0.0607 -- Training Accuracy: 